<a href="https://colab.research.google.com/github/27luiz/dadosSUS/blob/main/DadosSusCSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install dbc_reader
pip install pyspark

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for dbc_reader: filename=dbc_reader-0.1.2-py3-none-any.whl size=18497 sha256=b6e6f48f60fa70c98ab1904f1fa803d64678671f493ed9de1c1f083c229f050d
  Stored in directory: /root/.cache/pip/wheels/71/de/52/658cf7a77a062a6c35ce3eac5cd4f43502ddf7f60b5a5701b1
  Created wheel for dbf-reader: filename=dbf_reader-0.2.2-py3-none-any.whl size=7627 sha256=bf1f4660fec1dc555e340c7a98dfaa231eb3f2aed48408efa016a96141e3a307
  Stored in directory: /root/.cache/pip/wheels/ae/84/c4/e885b4d466946c4fd2473c1390a429ee4eec4cac53223413b8
Successfully built dbc_reader dbf-reader


In [10]:
from google.colab import auth
auth.authenticate_user()
# Inicializar o cliente do Google Cloud Storage
storage_client = storage.Client()

In [11]:
from google.cloud import storage
from dbc_reader import DbcReader
import pyspark.sql.functions as F
from pyspark.sql import SparkSession


In [12]:
# Caminho do arquivo DBC no Google Cloud Storage
gcs_bucket_name = "tabsus"
blob_name = "bruto/RDBA2301.dbc"
local_file_path = "/tmp/RDMG2301.dbc"

# Baixar o arquivo DBC localmente
bucket = storage_client.bucket(gcs_bucket_name)
blob = bucket.blob(blob_name)
blob.download_to_filename(local_file_path)

# Ler o arquivo DBC localmente usando dbc_reader
rows = [row for row in DbcReader(local_file_path)]

# Agora você pode processar as linhas conforme necessário
print(rows[0:10])

[{'UF_ZI': '290000', 'ANO_CMPT': '2023', 'MES_CMPT': '01', 'ESPEC': '01', 'CGC_HOSP': '', 'N_AIH': '2923101182970', 'IDENT': '1', 'CEP': '46430000', 'MUNIC_RES': '291170', 'NASC': '19810201', 'SEXO': '1', 'UTI_MES_IN': 0, 'UTI_MES_AN': 0, 'UTI_MES_AL': 0, 'UTI_MES_TO': 0, 'MARCA_UTI': '00', 'UTI_INT_IN': 0, 'UTI_INT_AN': 0, 'UTI_INT_AL': 0, 'UTI_INT_TO': 0, 'DIAR_ACOM': 0, 'QT_DIARIAS': 2, 'PROC_SOLIC': '0407030034', 'PROC_REA': '0407030034', 'VAL_SH': 751.13, 'VAL_SP': 245.99, 'VAL_SADT': 0.0, 'VAL_RN': 0.0, 'VAL_ACOMP': 0.0, 'VAL_ORTP': 0.0, 'VAL_SANGUE': 0.0, 'VAL_SADTSR': 0.0, 'VAL_TRANSP': 0.0, 'VAL_OBSANG': 0.0, 'VAL_PED1AC': 0.0, 'VAL_TOT': 997.12, 'VAL_UTI': 0.0, 'US_TOT': 191.75, 'DT_INTER': '20230117', 'DT_SAIDA': '20230119', 'DIAG_PRINC': 'K800', 'DIAG_SECUN': '0000', 'COBRANCA': '12', 'NATUREZA': '00', 'NAT_JUR': '1244', 'GESTAO': '2', 'RUBRICA': 0, 'IND_VDRL': '0', 'MUNIC_MOV': '290520', 'COD_IDADE': '4', 'IDADE': 41, 'DIAS_PERM': 2, 'MORTE': 0, 'NACIONAL': '010', 'NUM_PRO

In [13]:
# Inicializar a sessão do Spark
spark = SparkSession.builder \
    .appName("CreateDataFrameFromRows") \
    .getOrCreate()

# Converter as linhas para um DataFrame Spark
df = spark.createDataFrame(rows)

# Exibir o DataFrame Spark
df.show(10)

+--------+--------------------+-------+------+--------+--------------+--------+---------+---------+-----+-------+--------------+--------+---------+-------+----------+----------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+----------+----------+---------+---------+--------+--------+-----+-----+-------+------+------+----------+---------------+---------+---------+---------+--------+-----+-----+--------+--------+------------+------+---------+---------+--------+-----+---------+---------+--------+--------+--------+-------+----------+--------+-------------+----------+----------+----------+--------+-----+--------------------+-------+---------+--------+----+----------------+---------+--------+--------+--------+--------+--------+--------+--------+--------+--------+------+-------+----------+----------+----------+----------+----------+----------+----------+----------+---------+----------+--------+----------+------+--------+----------+----------+-------+-

In [14]:
# Coletar o DataFrame Spark para a máquina local
df_local = df.toPandas()

# Caminho para salvar o arquivo CSV localmente
local_csv_path = "/content/RDMG2301.csv"

# Salvar o DataFrame como um arquivo CSV localmente
df_local.to_csv(local_csv_path, index=False)

print("Arquivo CSV salvo localmente em:", local_csv_path)

# Transferir o arquivo CSV para o Google Cloud Storage
from google.cloud import storage

# Inicializar o cliente do Google Cloud Storage
storage_client = storage.Client()

# Especificar o caminho no Google Cloud Storage onde o arquivo CSV será salvo
gcs_bucket_name = "tabsus"
gcs_blob_name = "tratado/RDMG2301.csv"

# Fazer upload do arquivo local para o Google Cloud Storage
bucket = storage_client.bucket(gcs_bucket_name)
blob = bucket.blob(gcs_blob_name)
blob.upload_from_filename(local_csv_path)

print("Arquivo CSV transferido com sucesso para:", f"gs://{gcs_bucket_name}/{gcs_blob_name}")

Arquivo CSV salvo localmente em: /content/RDMG2301.csv
Arquivo CSV transferido com sucesso para: gs://tabsus/tratado/RDMG2301.csv
